In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#credit for this code: https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/
#this code creates a function that gets the browser to scroll down
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

In [3]:
chrome_path = "C:\Chromedrive\chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_path)


#this list represents different makeup categories to collect
categories = ['foundation-makeup',
             'eyeliner',
             'lipstick',
             'blush',
             'mascara']

final_list = []

for category in categories: #looping through the different categories
    
    page_num = 1 #the first page
    
    while True:
    
        #when running the following .get request in order for everything to work properly
        #the window needs to be popped open on the screen
        url = 'https://www.sephora.com/shop/'+ category + '?pageSize=300&currentPage=' + str(page_num)
        driver.get(url)
        time.sleep(1)
        try: #check to see if the page is empty   
            if driver.find_element_by_class_name('css-1kv743p').is_displayed():
                break
            
        except:
    
            #check to see if there is a pop up windew
            try:
                #exit the pop up window
                xpath = '//*[@id="modalDialog"]/button'
                btn = driver.find_element_by_xpath(xpath)
                btn.click()
                time.sleep(1)
            except:
                pass
    
            #as scrolling check if there is any more room to scroll
            old_len = 0
            
            while True:
                browser = scrollDown(driver, 20) #scroll down the page
                time.sleep(1) #give it time to load
                slug = driver.find_elements_by_class_name('css-ix8km1') #look for the urls of products
                
                new_len = len(slug)
                if old_len == new_len: #if the old length and new length are equal, the bottom of page was reached
                    break
                else:
                    old_len = new_len
                    
            #create list of URL      
            slugURL = []
            for a in slug:
                slugURL.append(a.get_attribute('href'))
                
            final_list.extend(slugURL)
            #adding to go to next page
            page_num += 1
    
#close the chrome
driver.close()
    

In [4]:
product_list = []
price_list = []
rating_list = []
img_list = []
category_list = []

for url in final_list:
    #print(url)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    time.sleep(10)
    
    try:
        
        product = soup.find_all('h1', class_='css-1wd4e6l')[0].text
        product_list.append(product)
        
        price = soup.find_all('div', class_='css-1865ad6')[0].string
        price_list.append(price)
        
        rating = soup.find_all('div', class_= 'css-jp4jy6')[0]['aria-label']
        rating_list.append(rating)
        
        img = 'https://www.sephora.com/' + soup.find_all('img', class_= 'css-1rovmyu')[0]['src']
        img_list.append(img)
        
        category = soup.find_all('a', class_='css-ti1lsh')[0].text
        category_list.append(category)
        
    except:
        pass


In [41]:
page = requests.get('https://www.sephora.com/product/color-block-high-impact-lipstick-P436270?skuId=2128692&icid2=products%20grid:p436270')

soup = BeautifulSoup(page.content, 'html.parser')
time.sleep(1)    

#print(soup.find_all('body'))

In [35]:
#page = requests.get('https://www.sephora.com/product/collectors-edition-le-marc-lip-creme-lipstick-blacquer-P410752')

#soup = BeautifulSoup(page.content, 'html.parser')
#time.sleep(1)    

#print('https://www.sephora.com/' + soup.find_all('img', class_= 'css-1rovmyu')[0]['src'])


https://www.sephora.com//productimages/sku/s1850254-main-zoom.jpg?imwidth=60


In [43]:
dict = {'Product': product_list,
        'Product Type': category_list,
        'Price': price_list,
        'Rating': rating_list,
        'Image URL': img_list
       }


In [16]:
sephora_df = pd.DataFrame.from_dict(dict, orient='index')



,0,1,2,3,4,5,6,7,8,9,...,913,914,915,916,917,918,919,920,921,922
Product,FENTY BEAUTY by RihannaPro Filt'r Soft Matte L...,FENTY BEAUTY by RihannaPro Filt'r Soft Matte P...,MAKE UP FOR EVERUltra HD Invisible Cover Found...,SEPHORA COLLECTIONBest Skin Ever Liquid Founda...,bareMineralsCOMPLEXION RESCUE™ Tinted Moisturi...,Rare Beauty by Selena GomezLiquid Touch Weight...,DiorDior Airflash Spray Foundation,MAKE UP FOR EVERReboot Active Care Revitalizin...,Laura MercierTinted Moisturizer Natural Skin P...,Charlotte TilburyAirbrush Flawless Longwear Fo...,...,None,None,None,None,None,None,None,None,None,None
Product Type,Foundation,BB & CC Cream,Foundation,Tinted Moisturizer,Foundation,Foundation,Foundation,Foundation,Foundation,Foundation,...,None,None,None,None,None,None,None,None,None,None
Price,$36.00,$36.00,$43.00,$20.00,$33.00,$29.00,$62.00,$39.00,$47.00,$44.00,...,None,None,None,None,None,None,None,None,None,None
Rating,4 stars,4.5 stars,4 stars,5 stars,4 stars,4 stars,4.5 stars,4 stars,4 stars,4 stars,...,None,None,None,None,None,None,None,None,None,None
URL,https://www.sephora.com/product/pro-filtr-soft...,https://www.sephora.com/product/ilia-super-ser...,https://www.sephora.com/product/luminous-silk-...,https://www.sephora.com/product/double-wear-st...,https://www.sephora.com/product/fenty-beauty-r...,https://www.sephora.com/product/ultra-hd-invis...,https://www.sephora.com/product/your-skin-but-...,https://www.sephora.com/product/sephora-collec...,https://www.sephora.com/product/nars-soft-matt...,https://www.sephora.com/product/natural-radian...,...,https://www.sephora.com/product/roen-beauty-ca...,https://www.sephora.com/product/sephora-favori...,https://www.sephora.com/product/lashes-to-die-...,https://www.sephora.com/product/saie-mascara-1...,https://www.sephora.com/product/too-faced-bett...,https://www.sephora.com/product/bestselling-ma...,https://www.sephora.com/product/too-faced-bett...,https://www.sephora.com/product/too-faced-bett...,https://www.sephora.com/product/too-faced-bett...,https://www.sephora.com/product/westman-atelie...
Image URL,https://www.sephora.com//productimages/sku/s21...,https://www.sephora.com//productimages/sku/s24...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s21...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s22...,...,None,None,None,None,None,None,None,None,None,None


In [17]:
sephora_df = sephora_df.transpose()

In [18]:
sephora_df.to_csv('sephora_df.csv', index=False)

In [19]:
path = 'sephora_df.csv'
sephora = pd.read_csv(path)

In [20]:
sephora_df.head()


,Product,Product Type,Price,Rating,URL,Image URL
0,FENTY BEAUTY by RihannaPro Filt'r Soft Matte L...,Foundation,$36.00,4 stars,https://www.sephora.com/product/pro-filtr-soft...,https://www.sephora.com//productimages/sku/s21...
1,FENTY BEAUTY by RihannaPro Filt'r Soft Matte P...,BB & CC Cream,$36.00,4.5 stars,https://www.sephora.com/product/ilia-super-ser...,https://www.sephora.com//productimages/sku/s24...
2,MAKE UP FOR EVERUltra HD Invisible Cover Found...,Foundation,$43.00,4 stars,https://www.sephora.com/product/luminous-silk-...,https://www.sephora.com//productimages/sku/s22...
3,SEPHORA COLLECTIONBest Skin Ever Liquid Founda...,Tinted Moisturizer,$20.00,5 stars,https://www.sephora.com/product/double-wear-st...,https://www.sephora.com//productimages/sku/s23...
4,bareMineralsCOMPLEXION RESCUE™ Tinted Moisturi...,Foundation,$33.00,4 stars,https://www.sephora.com/product/fenty-beauty-r...,https://www.sephora.com//productimages/sku/s21...


In [21]:
len(category_list)

226

In [32]:
test_list = final_list[:10]
test_list


['https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?icid2=products%20grid:p87985432',
 'https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?icid2=products%20grid:p455418',
 'https://www.sephora.com/product/luminous-silk-foundation-P393401?icid2=products%20grid:p393401',
 'https://www.sephora.com/product/double-wear-stay-in-place-makeup-P378284?icid2=products%20grid:p378284',
 'https://www.sephora.com/product/fenty-beauty-rihanna-pro-filt-r-soft-matte-powder-foundation-P466829?icid2=products%20grid:p466829',
 'https://www.sephora.com/product/ultra-hd-invisible-cover-foundation-P398321?icid2=products%20grid:p398321',
 'https://www.sephora.com/product/your-skin-but-better-cc-cream-spf-50-P411885?icid2=products%20grid:p411885',
 'https://www.sephora.com/product/sephora-collection-best-skin-ever-liquid-foundation-P467140?skuId=2326783&icid2=products%20grid:p467140',
 'https://www.sephora.com/product/nars-soft-matte-complete-foundation-P

In [33]:
for url in test_list:
    #print(url)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    time.sleep(1)
    
    try:
        
        product = soup.find_all('h1', class_='css-1wd4e6l')[0].text
        product_list.append(product)
        
        price = soup.find_all('div', class_='css-1865ad6')[0].string
        price_list.append(price)
        
        rating = soup.find_all('div', class_= 'css-jp4jy6')[0]['aria-label']
        rating_list.append(rating)
        
        img = 'https://www.sephora.com/' + soup.find_all('img', class_= 'css-1rovmyu')[0]['src']
        img_list.append(img)
        
        category = soup.find_all('a', class_='css-ti1lsh')[0].text
        category_list.append(category)
        
    except KeyError:
        pass

IndexError: list index out of range

In [45]:
dict = {'Product': product_list,
        'Product Type': category_list,
        'Price': price_list,
        'Rating': rating_list,
        'Image URL': img_list
       }

In [46]:
test_df = pd.DataFrame.from_dict(dict, orient='index')

test_df

,0,1,2,3,4,5,6,7,8,9,...,465,466,467,468,469,470,471,472,473,474
Product,FENTY BEAUTY by RihannaPro Filt'r Soft Matte L...,FENTY BEAUTY by RihannaPro Filt'r Soft Matte P...,MAKE UP FOR EVERUltra HD Invisible Cover Found...,SEPHORA COLLECTIONBest Skin Ever Liquid Founda...,bareMineralsCOMPLEXION RESCUE™ Tinted Moisturi...,Rare Beauty by Selena GomezLiquid Touch Weight...,DiorDior Airflash Spray Foundation,MAKE UP FOR EVERReboot Active Care Revitalizin...,Laura MercierTinted Moisturizer Natural Skin P...,Charlotte TilburyAirbrush Flawless Longwear Fo...,...,Westman AtelierEye Love You Volumizing Mascara,FENTY BEAUTY by RihannaPro Filt'r Soft Matte L...,ILIASuper Serum Skin Tint SPF 40 Foundation,Estée LauderDouble Wear Stay-in-Place Foundation,FENTY BEAUTY by RihannaPro Filt'r Soft Matte P...,bareMineralsCOMPLEXION RESCUE™ Tinted Moisturi...,Too FacedBorn This Way Foundation,LancômeTeint Idole Ultra Long Wear Foundation,DiorDior Airflash Spray Foundation,FENTY BEAUTY by RihannaPro Filt'r Soft Matte L...
Product Type,Foundation,BB & CC Cream,Foundation,Tinted Moisturizer,Foundation,Foundation,Foundation,Foundation,Foundation,Foundation,...,None,None,None,None,None,None,None,None,None,None
Price,$36.00,$36.00,$43.00,$20.00,$33.00,$29.00,$62.00,$39.00,$47.00,$44.00,...,$58.00,$36.00,$46.00,$43.00,$36.00,$33.00,$40.00,$47.00,$62.00,$36.00
Rating,4 stars,4.5 stars,4 stars,5 stars,4 stars,4 stars,4.5 stars,4 stars,4 stars,4 stars,...,No stars,4 stars,4 stars,4.5 stars,4.5 stars,4 stars,4 stars,4.5 stars,4.5 stars,4 stars
Image URL,https://www.sephora.com//productimages/sku/s21...,https://www.sephora.com//productimages/sku/s24...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s21...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s23...,https://www.sephora.com//productimages/sku/s22...,https://www.sephora.com//productimages/sku/s22...,...,None,None,None,None,None,None,None,None,None,None
